In [13]:
from cobra import Model, Reaction, Metabolite
from cobra.io import read_sbml_model
from energy_cycle import energy_generating_cycle_test

# read model for E. coli K12
k12 = read_sbml_model('../GEMs/iML1515.xml')

#read the universal model
uni = read_sbml_model('../GEMs/bigg_universe.xml')

No objective coefficients in model. Unclear what should be optimized


# MAM2 module
RA mod

In [14]:
MAM2 = k12.copy()

## Metabolites

In [15]:
#caffeic acid
caffeic_acid_c = uni.metabolites.get_by_id("34dhcinm_c")

#external caffeic acid
caffeic_acid_e = uni.metabolites.get_by_id("34dhcinm_e")

#caffeoyl CoA
caffcoa_c = uni.metabolites.get_by_id("caffcoa_c")

#3,4-Dihydroxyphenyllactic acid | Salvianic acid A | SAA
saa_c = Metabolite(
    'saa_c',
    formula='C9H10O5',
    name='3,4-Dihydroxyphenyllactic acid',
    compartment='c')

#external 3,4-Dihydroxyphenyllactic acid | Salvianic acid A | SAA
saa_e = Metabolite(
    'saa_e',
    formula='C9H10O5',
    name='3,4-Dihydroxyphenyllactic acid',
    compartment='e')

#rosmarinic acid
rosma_c = Metabolite(
    'rosma_c',
    formula='C18H16O8',
    name='Rosmarinic acid',
    compartment='c')

#external rosmarinic acid
rosma_e = Metabolite(
    'rosma_e',
    formula='C18H16O8',
    name='Rosmarinic acid',
    compartment='e')

#ATP
atp_c = k12.metabolites.get_by_id("atp_c")

#AMP
amp_c = k12.metabolites.get_by_id("amp_c")

#Diphosphate
ppi_c = k12.metabolites.get_by_id("ppi_c")

#CoA
coa_c = k12.metabolites.get_by_id("coa_c")

## Reactions

In [16]:
#caffeic acid transport reaction
CAFFAt = Reaction('CAFFAt')
CAFFAt.name = 'caffa transport'
CAFFAt.subsystem = 'RA module'
CAFFAt.lower_bound = -1000  # This is the default
CAFFAt.upper_bound = 1000  # This is the default
CAFFAt.add_metabolites({
    caffeic_acid_c: -1.0,
    caffeic_acid_e: 1.0
})

MAM2.add_reactions([CAFFAt])

In [17]:
CAFFCOA = uni.reactions.get_by_id("CAFFCOA")

MAM2.add_reactions([CAFFCOA])

In [18]:
#SAA transport reaction
SAAt = Reaction('SAAt')
SAAt.name = 'caffa transport'
SAAt.subsystem = 'RA module'
SAAt.lower_bound = -1000  # This is the default
SAAt.upper_bound = 1000  # This is the default
SAAt.add_metabolites({
    saa_c: -1.0,
    saa_e: 1.0
})

MAM2.add_reactions([SAAt])

In [19]:
#salvianic acid A + caffeoyl CoA -> Rosmarinic acid
RAS = Reaction('RAS')
RAS.name = 'rosmarinic acid synthase'
RAS.subsystem = 'RA module'
RAS.lower_bound = -1000  # This is the default
RAS.upper_bound = 1000  # This is the default
#Caffeoyl-CoA + 3-(3,4-Dihydroxyphenyl)lactate <=> CoA + Rosmarinate
RAS.add_metabolites({
    caffcoa_c: -1.0,
    saa_c: -1.0,
    coa_c: 1.0,
    rosma_c: 1.0
})
#RAS.gene_reaction_rule = '(ras)' #synthetic 4CL gene

MAM2.add_reactions([RAS])

In [20]:
#RA transport reaction
RAt = Reaction('RAt')
RAt.name = 'caffa transport'
RAt.subsystem = 'RA module'
RAt.lower_bound = -1000  # This is the default
RAt.upper_bound = 1000  # This is the default
RAt.add_metabolites({
    rosma_c: -1.0,
    rosma_e: 1.0
})

MAM2.add_reactions([RAt])

In [21]:
#exhange reactions for SA, caffa, and RA

MAM2.metabolites.get_by_id("34dhcinm_e").compartment = 'e' #set compartment to be 'e' rather than 'C_e'

MAM2.add_boundary(MAM2.metabolites.get_by_id("34dhcinm_e"), type="exchange")
MAM2.add_boundary(MAM2.metabolites.get_by_id("saa_e"), type="exchange")
MAM2.add_boundary(MAM2.metabolites.get_by_id("rosma_e"), type="exchange")

Reaction identifier,EX_rosma_e
Name,Rosmarinic acid exchange
Memory address,0x131a3be20
Stoichiometry,rosma_e <=> Rosmarinic acid <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [22]:
#so that cell cannot eat rosmarinic acid
MAM2.reactions.EX_rosma_e.lower_bound = 0

## Testing

In [15]:
energy_generating_cycle_test(MAM2)

ATPM flux: 0.0


,fluxes,reduced_costs
CYTDK2,0.0,-0.293564
XPPT,0.0,10.161276
HXPRT,0.0,10.161276
NDPK5,0.0,-1.919362
SHK3Dr,0.0,2.000000
...,...,...
RAS,0.0,-2.000000
RAt,0.0,2.000000
EX_34dhcinm_e,0.0,-2.000000
EX_saa_e,0.0,-2.000000


In [17]:
from cobra.flux_analysis import flux_variability_analysis

MAM2.metabolites.rosma_c.summary(fva=0.95)

Percent,Flux,Range,Reaction,Definition
,0,[0; 6.661],RAS,caffcoa_c + saa_c <=> coa_c + rosma_c
Percent,Flux,Range,Reaction,Definition
,0,[-6.661; 0],RAt,rosma_c <=> rosma_e


## Export model to SBML

In [24]:
#updtate the model ID

MAM2.id = 'MAM2'

In [26]:
from cobra.io import write_sbml_model

write_sbml_model(MAM2, "../GEMs/MAM2.xml")